In [12]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from keras.callbacks import TensorBoard
import random


In [8]:
EPOCHS = 10
IMG_SIZE = 200
CATEGORIES = ["Rock", "Paper", "Scissors"]
ROOT_DIR = "/mnt/c/Users/Satyamedh/PycharmProjects/WebcamRockPaperScissors" # use wsl mounting path cuz fukin tensorflow doesn't work on windows with gpu


In [15]:
# Load the data stored in X.pickle and y.pickle
import pickle

X = pickle.load(open(os.path.join(ROOT_DIR, "X.pickle"), "rb"))
y = pickle.load(open(os.path.join(ROOT_DIR, "y.pickle"), "rb"))
y = np.array(y)

In [10]:
# Normalize the data
X = X/255.0

In [6]:
# Data is already balanced, we can move on to the model. DO NOT SHUFFLE THE DATA AS IT IS ALIGNED WITH THE LABELS

In [13]:
# model is a simple 128x3 CNN

model = Sequential()

model.add(Conv2D(128, (3, 3), input_shape=X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dense(3))
model.add(Activation("softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"]) # since we are using sparse_categorical_crossentropy, we don't need to one-hot encode the labels yay (I think)



In [16]:
# Train the model!
tensorboard = TensorBoard(log_dir=os.path.join(ROOT_DIR, "logs"))
model.fit(X, y, batch_size=32, epochs=EPOCHS, validation_split=0.1, callbacks=[tensorboard])

2024-02-18 01:21:28.045361: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/10


2024-02-18 01:21:28.739787: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-18 01:21:29.101346: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-18 01:21:29.771696: I external/local_xla/xla/service/service.cc:168] XLA service 0x951b230 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-18 01:21:29.771727: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-02-18 01:21:29.784479: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708199489.869121    1687 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


9/9 [==============================] - 7s 283ms/step - loss: 152.1206 - accuracy: 0.3222 - val_loss: 11.4135 - val_accuracy: 0.2667
Epoch 2/10
9/9 [==============================] - 0s 52ms/step - loss: 3.2314 - accuracy: 0.4370 - val_loss: 1.2497 - val_accuracy: 0.3000
Epoch 3/10
9/9 [==============================] - 0s 51ms/step - loss: 1.3746 - accuracy: 0.4333 - val_loss: 1.2273 - val_accuracy: 0.3667
Epoch 4/10
9/9 [==============================] - 0s 52ms/step - loss: 1.0642 - accuracy: 0.4593 - val_loss: 1.1064 - val_accuracy: 0.3667
Epoch 5/10
9/9 [==============================] - 0s 52ms/step - loss: 1.0382 - accuracy: 0.5222 - val_loss: 1.0038 - val_accuracy: 0.5000
Epoch 6/10
9/9 [==============================] - 0s 52ms/step - loss: 0.9461 - accuracy: 0.5778 - val_loss: 0.9291 - val_accuracy: 0.5667
Epoch 7/10
9/9 [==============================] - 0s 51ms/step - loss: 0.8473 - accuracy: 0.7222 - val_loss: 1.0617 - val_accuracy: 0.3667
Epoch 8/10
9/9 [==================

In [17]:
model.save(os.path.join(ROOT_DIR, "rock_paper_scissors_FIRST_ONE_EVER.model"))

INFO:tensorflow:Assets written to: /mnt/c/Users/Satyamedh/PycharmProjects/WebcamRockPaperScissors/rock_paper_scissors_FIRST_ONE_EVER.model/assets


INFO:tensorflow:Assets written to: /mnt/c/Users/Satyamedh/PycharmProjects/WebcamRockPaperScissors/rock_paper_scissors_FIRST_ONE_EVER.model/assets


In [23]:
# Let's test the model
def prepare(filepath):
    IMG_SIZE = 200
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

IMG = os.path.join(ROOT_DIR, "data/Paper/34.jpg") # A random image from the dataset. prolly should use a different image that the model hasn't seen before but I'll do that tomorrow
prediction = model.predict([prepare(IMG)])
print(CATEGORIES[np.argmax(prediction)])

1/1 [==============================] - 0s 12ms/step
Scissors


In [22]:
# yea it sucks. I'll try to get a better dataset tomorrow. goodnight